### File: Exploratory_Data_Analysis.ipynb - CineMatch

- Contributor: Sudesh Kumar Santhosh Kumar
- Email: santhosh@usc.edu
- GitHub: [Sudesh Kumar](https://github.com/SudeshKumarSanthosh)
- Date: 4th December, 2023

### Description:

This Jupyter Notebook serves as the backbone of the CineMatch project's initial data exploration stage. Within these pages lies a comprehensive analysis of the MovieLens dataset, a treasure trove of movie-related information. The notebook meticulously dissects various facets of the dataset, including plots, genres, production details, financial metrics, and popularity scores.

`Key highlights of this exploratory analysis include:`

* Detailed statistical summaries and visualizations that uncover underlying patterns and trends in movie data.

* A thorough examination of the dataset's structure and content, laying the groundwork for the application of advanced deep learning and transformer technologies in CineMatch.

* Insightful correlations and comparisons between different cinematic attributes, offering a multifaceted understanding of what shapes movie popularity and viewer preferences.

* Innovative approaches to data cleaning and preprocessing, ensuring the integrity and usability of the dataset for further modeling.

This exploratory data analysis is not just a preliminary step but a crucial foundation for the sophisticated recommendation algorithms that will drive CineMatch. It aims to provide a clear, data-informed picture of the diverse landscape of cinema, setting the stage for a personalized and enriched movie selection experience.

## Part 1 - Installation and Dataset Loading
-----

### S1. Setting up the notebook

### 1.1. Importing all necessary packages

In [ ]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
